In [5]:
import cv2 as cv
import numpy as np
import scipy
import pickle
import random
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial.distance import euclidean,cosine



In [6]:
directory='/Users/maddisiddart/Desktop/train'
img_dirs=os.listdir(directory)[:10]

In [7]:
img_dirs

['gerenuk',
 'hawksbill',
 'headphone',
 'ant',
 'butterfly',
 'lamp',
 'strawberry',
 'water_lilly',
 'chandelier',
 'dragonfly']

In [8]:
#finding descriptors
desc=[]
X=[]
sift = cv.xfeatures2d.SIFT_create()
final_images=[]
for img_dir in img_dirs:
    images = os.listdir(directory+'/'+img_dir)
    for img_name in images:
        img_path=os.path.join(directory,img_dir,img_name)
        img=cv.imread(img_path)
        kps=sift.detect(img)
        kp1, des1 = sift.compute(img, kps)
        try:
            if (des1.all!=None):
                final_images.append(img_path)
                desc.append([])
                desc[-1].append(des1)
                for d in des1:
                    X.append(d)
        except:
            pass

In [9]:
output = open('descriptor.pkl', 'wb')
pickle.dump(desc,output,pickle.HIGHEST_PROTOCOL) 

In [10]:
out = open('images.pkl', 'wb')
pickle.dump(final_images,out,pickle.HIGHEST_PROTOCOL)

In [12]:
len(final_images)

617

In [13]:
#clustering of key points to make dictionary using k means
dictionary_size=2000
kmeans = MiniBatchKMeans(init='k-means++',n_clusters=dictionary_size,batch_size=500,random_state=0).fit(X)
centers=kmeans.cluster_centers_
print("clustered")

/Users/maddisiddart/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=1500 should be larger than k=2000. Setting it to 3*k
  init_size=init_size)
/Users/maddisiddart/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=1500 should be larger than k=2000. Setting it to 3*k
  init_size=init_size)
/Users/maddisiddart/opt/anaconda3/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=1500 should be larger than k=2000. Setting it to 3*k
  init_size=init_size)


clustered


In [14]:
cen = open('clustered.pkl', 'wb')
pickle.dump(centers,cen,pickle.HIGHEST_PROTOCOL)

In [15]:
#flann matcher to match key points to words in dictionary
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50) 

flann = cv.FlannBasedMatcher(index_params,search_params)
arr=[]
for i in range(len(desc)):
    matches = flann.knnMatch(np.asarray(desc[i][0],np.float32),np.asarray(centers,np.float32),k=1)
    #for j in range(len(matches)):
        #arr[-1].append(matches[j][0].trainIdx)
    arr.append([matches[j][0].trainIdx for j in range(len(matches))])


In [27]:
'''
#creating a histogram
doc=[]
for i in range(len(arr)): 
    hist=[0]*(dictionary_size)
    for j in range(0,dictionary_size):
        l = len((np.where(np.array(arr[i]) == j))[0])
        hist[j] = l
    doc.append(hist)
print("histogram with frequency of words in an image done (method 1)")
'''

'\n#creating a histogram\ndoc=[]\nfor i in range(len(arr)): \n    hist=[0]*(dictionary_size)\n    for j in range(0,dictionary_size):\n        l = len((np.where(np.array(arr[i]) == j))[0])\n        hist[j] = l\n    doc.append(hist)\nprint("histogram with frequency of words in an image done (method 1)")\n'

In [16]:
doc_word=[]
for i in range(len(arr)):
    hist=[0]*dictionary_size
    c={}
    for j in range(len(arr[i])):
        if (arr[i][j] in c):
            c[arr[i][j]]+=1
        else:
            c[arr[i][j]]=1
    for k in c:
        hist[k]=c[k]
    doc_word.append(hist)
print("histogram with frequency of words in an image done (method 2)")       
    

histogram with frequency of words in an image done (method 2)


In [17]:
his = open('histogram.pkl', 'wb')
pickle.dump(doc_word,his,pickle.HIGHEST_PROTOCOL)

In [18]:
print(len(doc_word))

617


In [19]:
# topic modelling 
ld = LatentDirichletAllocation(n_components=20,learning_method='online',max_iter=100)
ld.fit(doc_word)
topic_dist=ld.transform(doc_word)


In [20]:
t_dist = open('topic_dist.pkl', 'wb')
pickle.dump(topic_dist,t_dist,pickle.HIGHEST_PROTOCOL)

In [22]:
topic_dist


array([[1.54368077e-01, 1.30984244e-01, 4.04111505e-01, 3.49268947e-04,
        3.10186905e-01],
       [8.35755804e-01, 3.18097228e-04, 7.05895690e-02, 3.11265121e-04,
        9.30252643e-02],
       [8.19619577e-01, 6.10948015e-02, 9.07400878e-02, 4.45703655e-04,
        2.80998297e-02],
       ...,
       [4.28129297e-01, 2.62951611e-01, 3.08036913e-01, 4.35053626e-04,
        4.47125990e-04],
       [1.22792028e-01, 3.39996427e-01, 1.24134723e-01, 2.62626758e-04,
        4.12814195e-01],
       [2.66021657e-04, 2.65817444e-04, 9.98940852e-01, 2.61201235e-04,
        2.66108032e-04]])

In [26]:
#querying an image 
query_image_path='/Users/maddisiddart/Desktop/train/butterfly/image_0001.jpg'
query_img=cv.imread(query_image_path)
query_img_kps=sift.detect(query_img)
query_kp, query_des = sift.compute(query_img,query_img_kps)

FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50) 

flann = cv.FlannBasedMatcher(index_params,search_params)
query_arr=[]
matches = flann.knnMatch(np.asarray(query_des,np.float32),np.asarray(centers,np.float32),k=1)
query_arr.append([matches[j][0].trainIdx for j in range(len(matches))])

dictionary_size=2000
q={}
query_hist=[0]*dictionary_size
for j in range(len(query_arr[0])):
    if (query_arr[0][j] in q):
        q[query_arr[0][j]]+=1
    else:
        q[query_arr[0][j]]=1
for k in q:
    query_hist[k]=q[k]

query_hist=np.array(query_hist).reshape(1,-1)

query_topic = ld.transform(query_hist)

euc=[]
for i in range(len(topic_dist)):
    euc.append([final_images[i],euclidean(query_topic[0],topic_dist[i])])
    
img_sorted = sorted(euc, key=lambda x: x[1])


In [27]:
img_sorted

[['/Users/maddisiddart/Desktop/train/butterfly/image_0001.jpg', 0.0],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0031.jpg',
  0.012543512389350682],
 ['/Users/maddisiddart/Desktop/train/dragonfly/image_0009.jpg',
  0.34249270052093544],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0017.jpg',
  0.37298264623028543],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0035.jpg',
  0.49555591070038246],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0082.jpg',
  0.5432796771956419],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0027.jpg',
  0.5616173974387609],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0049.jpg',
  0.580145436163895],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0025.jpg',
  0.596683594401238],
 ['/Users/maddisiddart/Desktop/train/butterfly/image_0055.jpg',
  0.6472071625867017],
 ['/Users/maddisiddart/Desktop/train/strawberry/image_0005.jpg',
  0.655610895261339],
 ['/Users/maddisiddart/Desktop/train/strawberry/image_003

In [26]:
for i in range(10):
    out = cv.imread(img_sorted[i][0],1)
    cv.imshow(str(i),out)

In [ ]:
_